# ARモデルに従う時系列の次時刻予測を、sklearnの回帰で行う。（最小二乗法）

In [2]:
import argparse
import os
path = os.getcwd()
path=path[:path.find('estimate-innovation')+20]
# No = (os.path.basename(__file__))[-4]
No = "0" # notebook用
# print('実験No.'+No)
import warnings
warnings.simplefilter('ignore')# 警告を非表示
import numpy as np
np.set_printoptions(precision=4, floatmode='fixed', suppress=True)
import matplotlib.pyplot as plt
from scipy import stats
import math
import sys
sys.path.append(path)
import random
import time
import statsmodels.api as sm
from scipy.stats import norm
import japanize_matplotlib
from scipy.stats import gaussian_kde

import torchvision.transforms as transforms
from torchvision.utils import save_image

import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.autograd import Variable
# 人工データを生成してくれる機械が置いてあるところ
import tsModel
# 学習用のニューラルネットが置いてあるところ
import models
# p-Wasserstein距離の関数
import Wasserstein
path

'/Users/ryuichi/Documents/research/estimate-innovation/'

### ARモデルに従う時系列の作成

In [3]:
phi_a = [1.0, -0.5, 0.7, -0.4]
phi_b = [0]
p = len(phi_a)
q = len(phi_b)
dataSeed=5
N = 1000
data = tsModel.ARIMA(a=phi_a, b=phi_b, N=N, random_seed=dataSeed, randomness='normal')
inno = tsModel.ARIMA(a=phi_a, b=phi_b, N=N, random_seed=dataSeed, randomness='normal', return_innovation=True)

### 推定に用いる$\hat p$を決定

In [4]:
hat_p = 4

### 学習データとvalidationデータを分ける（9:1）

In [5]:
trainData = data[:N//10*9]
valData = data[N//10*9:]

### データセットの作成

In [11]:
def make_XY(x:np.ndarray, p:int):
    X = [x[i:i+p] for i in range(len(x)-p)]
    Y = [x[i+p] for i in range(len(x)-p)]
    return X, Y
X,Y=make_XY(x=trainData, p=hat_p)

In [25]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression() # 線形回帰モデルを作成
X =np.array(X) # 説明変数
Y = Y # 目的変数
lr.fit(X, Y) # 学習
a = lr.intercept_ # 学習した切片
b = lr.coef_ # 学習した傾き
np.flip(b)

array([ 0.9997, -0.5038,  0.7098, -0.4041])

In [24]:
phi_a

[1.0, -0.5, 0.7, -0.4]